# Tree Scout
In this notebook we will use the Global Forest Watch API of the past decade to predict future deforestation

In [21]:
# Import necessary packages
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json
import warnings
warnings.filterwarnings('ignore')

# For environment variables
from dotenv import load_dotenv
import os

# For machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load environment variables from .env file
load_dotenv()

print("All packages imported successfully!")

All packages imported successfully!


In [ ]:
# Collect data from Global Forest Watch API with automatic token refresh
# GFW API endpoint for tree cover loss data
# Data collection limited to years up to 2024 for model testing

class GFWAuthManager:
    """Manages authentication and token refresh for GFW API"""
    
    def __init__(self, username=None, password=None, api_key=None):
        """
        Initialize the auth manager
        
        Parameters:
        - username: GFW username (or set GFW_USERNAME in .env file)
        - password: GFW password (or set GFW_PASSWORD in .env file)
        - api_key: GFW API key (or set GFW_API_KEY in .env file)
        """
        self.username = username or os.getenv('GFW_USERNAME')
        self.password = password or os.getenv('GFW_PASSWORD')
        self.api_key = api_key or os.getenv('GFW_API_KEY')
        self.token = None
        self.token_expiry = None
        self.auth_url = "https://data-api.globalforestwatch.org/auth/token"
    
    def get_token(self):
        """Get a valid bearer token, refreshing if necessary"""
        if self.token and self.token_expiry and datetime.now() < self.token_expiry:
            print("Using existing bearer token")
            return self.token
        
        print("Fetching new bearer token...")
        return self._refresh_token()
    
    def _refresh_token(self):
        """Refresh the authentication bearer token"""
        if not self.username or not self.password:
            print("Warning: No credentials provided. Set GFW_USERNAME and GFW_PASSWORD in .env file.")
            return None
        
        try:
            response = requests.post(
                self.auth_url,
                headers={'Content-Type': 'application/x-www-form-urlencoded'},
                data={'username': self.username, 'password': self.password},
                timeout=30
            )
            
            if response.status_code == 200:
                result = response.json()
                # Handle nested response structure
                if 'data' in result and 'access_token' in result['data']:
                    self.token = result['data']['access_token']
                elif 'access_token' in result:
                    self.token = result['access_token']
                else:
                    print("Unexpected response structure:", result)
                    return None
                
                # Set expiry (default to 1 hour if not provided)
                expires_in = result.get('expires_in', 3600)
                self.token_expiry = datetime.now() + timedelta(seconds=expires_in - 60)  # 60s buffer
                
                print(f"Bearer token obtained successfully. Expires at: {self.token_expiry.strftime('%Y-%m-%d %H:%M:%S')}")
                return self.token
            else:
                print(f"Authentication failed: {response.status_code}")
                print(f"Response: {response.text[:500]}")
                return None
        except Exception as e:
            print(f"Error refreshing token: {str(e)}")
            return None

def fetch_gfw_data(start_year=2010, end_year=2024):
    """
    Fetch tree cover loss data from Global Forest Watch using Data API
    
    Parameters:
    - start_year: Starting year for data collection (default: 2010)
    - end_year: Ending year for data collection (default: 2024, reserving 2025 for testing)
    
    Returns:
    - DataFrame with tree cover statistics from GFW Data API
    """
    
    print(f"Fetching Global Forest Watch data from {start_year} to {end_year}...")
    print("Using GFW Data API with continental US geometry...\n")
    
    import time
    
    # Continental United States bounding box geometry
    # Approximate bounds: West: -125°, East: -66°, South: 24°, North: 49°
    continental_us_geometry = {
        "type": "Polygon",
        "coordinates": [[
            [-125.0, 24.0],   # Southwest corner
            [-66.0, 24.0],    # Southeast corner
            [-66.0, 49.0],    # Northeast corner
            [-125.0, 49.0],   # Northwest corner
            [-125.0, 24.0]    # Close the polygon
        ]]
    }
    
    # Get API key from environment
    api_key = os.getenv('GFW_API_KEY')
    if not api_key:
        print("⚠ Warning: No GFW_API_KEY found in .env file")
        return pd.DataFrame()
    
    print(f"Using API key: {api_key[:20]}...")
    print("Querying GFW Data API with continental US geometry...")
    
    base_url = "https://data-api.globalforestwatch.org/dataset/umd_tree_cover_loss/latest"
    
    data_records = []
    
    try:
        # Query the Data API with geometry
        # SQL query to get tree cover loss aggregated by year for continental US
        sql_query = f"""
            SELECT 
                umd_tree_cover_loss__year as year,
                SUM(umd_tree_cover_loss__ha) as tree_cover_loss_ha
            FROM data
            WHERE umd_tree_cover_loss__year >= {start_year} 
                AND umd_tree_cover_loss__year <= {end_year}
            GROUP BY umd_tree_cover_loss__year
            ORDER BY umd_tree_cover_loss__year
        """
        
        query_url = f"{base_url}/query"
        
        # GFW Data API requires geometry as JSON in the request body for POST
        # Or we can use geostore_id - let's try with geometry first
        headers = {
            'x-api-key': api_key,
            'Content-Type': 'application/json'
        }
        
        payload = {
            'sql': sql_query,
            'geometry': continental_us_geometry
        }
        
        print(f"Sending POST request to: {query_url}")
        print(f"Query: {sql_query[:100]}...")
        
        response = requests.post(query_url, json=payload, headers=headers, timeout=60)
        
        print(f"Response status: {response.status_code}")
        
        if response.status_code == 200:
            result = response.json()
            print("✓ Query successful!")
            
            if 'data' in result and len(result['data']) > 0:
                print(f"✓ Retrieved {len(result['data'])} records")
                
                # Convert to our format
                for record in result['data']:
                    data_records.append({
                        'iso': 'USA',
                        'country': 'United States (Continental)',
                        'umd_tree_cover_loss__year': record.get('year'),
                        'tree_cover_loss_ha': record.get('tree_cover_loss_ha', 0)
                    })
            else:
                print("⚠ No data returned from query")
                print(f"Response: {json.dumps(result, indent=2)[:500]}")
        else:
            print(f"⚠ Query failed with status {response.status_code}")
            print(f"Response: {response.text[:500]}")
            
    except Exception as e:
        print(f"✗ Error querying GFW Data API: {str(e)}")
        import traceback
        traceback.print_exc()
    
    # Convert to DataFrame
    if data_records:
        df = pd.DataFrame(data_records)
        print(f"\n✓ Dataset created from GFW Data API")
        print(f"  • Records: {len(data_records)}")
        print(f"  • Years: {start_year}-{end_year}")
        print(f"  • Shape: {df.shape}")
        print(f"  • Columns: {df.columns.tolist()}")
        return df
    else:
        print("\n✗ No data retrieved from GFW Data API")
        print("Please check:")
        print("  1. API key is valid")
        print("  2. Geometry is correct")
        print("  3. Network connection")
        return pd.DataFrame()

# Initialize authentication manager (reads from .env file)
auth_manager = GFWAuthManager()

# Fetch the data (limited to 2024) with authentication
df_forest_data = fetch_gfw_data(start_year=2001, end_year=2024, auth_manager=auth_manager)
# Fetch the data (limited to 2024) with authentication
# Display basic information
print("\n" + "="*50)
print("Data Summary:")
print("="*50)
print(df_forest_data.head())
print(f"\nDate range: 2001-2024 (2025 data reserved for testing)")
print(f"Total records: {len(df_forest_data)}")

TypeError: fetch_gfw_data() got an unexpected keyword argument 'auth_manager'

In [23]:
# Test GFW API connectivity
# This cell tests if we can connect to the GFW API at all

print("Testing GFW API Connectivity...")
print("="*60)

import requests
import time

# Test 1: Check if the API base URL is reachable
print("\n1. Testing base API endpoint...")
try:
    response = requests.get("https://production-api.globalforestwatch.org", timeout=10)
    print(f"   Status: {response.status_code}")
    if response.status_code == 200:
        print("   ✓ Base API is reachable")
    else:
        print(f"   ⚠ Base API returned: {response.status_code}")
except Exception as e:
    print(f"   ✗ Error: {str(e)}")

# Test 2: Try a simple country endpoint (Brazil)
print("\n2. Testing country data endpoint (Brazil)...")
try:
    url = "https://production-api.globalforestwatch.org/v2/umd-loss-gain/admin/BRA"
    response = requests.get(url, timeout=10)
    print(f"   Status: {response.status_code}")
    
    if response.status_code == 200:
        data = response.json()
        print("   ✓ Successfully retrieved data!")
        print(f"   Response keys: {list(data.keys())}")
        if 'data' in data:
            print(f"   Data keys: {list(data['data'].keys())}")
    elif response.status_code == 503:
        print("   ⚠ Service unavailable (503) - API may be down or rate-limited")
    else:
        print(f"   ⚠ Unexpected status: {response.status_code}")
        print(f"   Response: {response.text[:200]}")
except Exception as e:
    print(f"   ✗ Error: {str(e)}")

# Test 3: Try with a small delay and retry
print("\n3. Testing with retry after delay...")
time.sleep(3)
try:
    url = "https://production-api.globalforestwatch.org/v2/umd-loss-gain/admin/USA"
    response = requests.get(url, timeout=10)
    print(f"   Status: {response.status_code}")
    
    if response.status_code == 200:
        print("   ✓ Retry successful! API is working.")
    elif response.status_code == 503:
        print("   ⚠ Still getting 503 - API appears to be down")
        print("   Recommendation: Use fallback sample data for now")
    else:
        print(f"   Status: {response.status_code}")
except Exception as e:
    print(f"   ✗ Error: {str(e)}")

# Test 4: Check alternative GFW endpoints
print("\n4. Testing alternative GFW data sources...")
alternative_urls = [
    "https://data-api.globalforestwatch.org",
    "https://www.globalforestwatch.org/api",
]

for alt_url in alternative_urls:
    try:
        response = requests.get(alt_url, timeout=5)
        print(f"   {alt_url}: {response.status_code}")
    except Exception as e:
        print(f"   {alt_url}: ✗ {str(e)[:50]}")

print("\n" + "="*60)
print("API Test Complete")
print("="*60)

Testing GFW API Connectivity...

1. Testing base API endpoint...
   Status: 200
   ✓ Base API is reachable

2. Testing country data endpoint (Brazil)...
   Status: 200
   ✓ Base API is reachable

2. Testing country data endpoint (Brazil)...
   Status: 503
   ⚠ Service unavailable (503) - API may be down or rate-limited

3. Testing with retry after delay...
   Status: 503
   ⚠ Service unavailable (503) - API may be down or rate-limited

3. Testing with retry after delay...
   Status: 503
   ⚠ Still getting 503 - API appears to be down
   Recommendation: Use fallback sample data for now

4. Testing alternative GFW data sources...
   Status: 503
   ⚠ Still getting 503 - API appears to be down
   Recommendation: Use fallback sample data for now

4. Testing alternative GFW data sources...
   https://data-api.globalforestwatch.org: 200
   https://data-api.globalforestwatch.org: 200
   https://www.globalforestwatch.org/api: 404

API Test Complete
   https://www.globalforestwatch.org/api: 404


## Direct API Test with PowerShell

Run these commands in your PowerShell terminal to test the GFW API directly (outside of Python code):

### Test 1: Simple GET request to Brazil endpoint
```powershell
Invoke-RestMethod -Uri "https://production-api.globalforestwatch.org/v2/umd-loss-gain/admin/BRA" -Method Get | ConvertTo-Json -Depth 3
```

### Test 2: Using curl.exe (native curl, not PowerShell alias)
```powershell
curl.exe -X GET "https://production-api.globalforestwatch.org/v2/umd-loss-gain/admin/BRA" -H "Accept: application/json"
```

### Test 3: Check response headers for rate limiting info
```powershell
$response = Invoke-WebRequest -Uri "https://production-api.globalforestwatch.org/v2/umd-loss-gain/admin/BRA" -Method Get
Write-Host "Status Code: $($response.StatusCode)"
Write-Host "Headers:"
$response.Headers
Write-Host "`nContent (first 500 chars):"
$response.Content.Substring(0, [Math]::Min(500, $response.Content.Length))
```

### Test 4: Try a different country (USA)
```powershell
Invoke-RestMethod -Uri "https://production-api.globalforestwatch.org/v2/umd-loss-gain/admin/USA" | ConvertTo-Json -Depth 2
```

**Copy and paste these commands into your PowerShell terminal to test if the API responds differently outside Python.**

In [ ]:
# Explore alternative GFW API endpoints to find working services
print("Searching for alternative GFW API endpoints...")
print("="*60)

import requests
import json

# Test various GFW API endpoints to find working ones
test_endpoints = [
    # Try v3 APIs
    ("v3 Forest Change", "https://production-api.globalforestwatch.org/v3/forest-change/glad/loss/BRA"),
    ("v3 Widgets", "https://production-api.globalforestwatch.org/v3/widget/treeloss/country/BRA"),
    ("v3 Analysis", "https://production-api.globalforestwatch.org/v3/analysis/country/BRA"),
    
    # Try v1 APIs (older but might still work)
    ("v1 Forest Stats", "https://production-api.globalforestwatch.org/v1/forest-change/umd-loss-gain/admin/BRA"),
    ("v1 Query", "https://production-api.globalforestwatch.org/v1/query/umd-loss-gain?iso=BRA"),
    
    # Try Data API datasets
    ("Data API Datasets", "https://data-api.globalforestwatch.org/dataset"),
    ("Data API Tree Cover Loss", "https://data-api.globalforestwatch.org/dataset/umd_tree_cover_loss/latest"),
    
    # Try newer analysis endpoints
    ("Analysis Glad", "https://production-api.globalforestwatch.org/analysis/glad/BRA"),
    ("Analysis TC Loss", "https://production-api.globalforestwatch.org/analysis/tc-loss/BRA"),
]

working_endpoints = []

for name, url in test_endpoints:
    try:
        response = requests.get(url, timeout=10)
        status = response.status_code
        
        if status == 200:
            print(f"✓ {name}")
            print(f"  URL: {url}")
            print(f"  Status: {status}")
            try:
                data = response.json()
                print(f"  Keys: {list(data.keys())[:5]}")
                working_endpoints.append((name, url, data))
            except:
                print(f"  Content length: {len(response.text)}")
            print()
        elif status == 404:
            print(f"✗ {name}: Not Found (404)")
        elif status == 503:
            print(f"⚠ {name}: Service Unavailable (503)")
        else:
            print(f"⚠ {name}: HTTP {status}")
            
    except Exception as e:
        print(f"✗ {name}: {str(e)[:50]}")

print("="*60)
print(f"\nFound {len(working_endpoints)} working endpoint(s)")

if working_endpoints:
    print("\nWorking endpoints details:")
    for i, (name, url, data) in enumerate(working_endpoints, 1):
        print(f"\n{i}. {name}")
        print(f"   URL: {url}")
        print(f"   Sample data structure:")
        print(f"   {json.dumps(data, indent=2)[:500]}...")


Searching for alternative GFW API endpoints...
⚠ v3 Forest Change: HTTP 403
⚠ v3 Forest Change: HTTP 403
⚠ v3 Widgets: HTTP 403
⚠ v3 Widgets: HTTP 403
⚠ v3 Analysis: HTTP 403
⚠ v3 Analysis: HTTP 403
⚠ v1 Forest Stats: HTTP 403
⚠ v1 Forest Stats: HTTP 403
⚠ v1 Query: HTTP 400
⚠ v1 Query: HTTP 400
✗ Data API Datasets: Not Found (404)
✗ Data API Datasets: Not Found (404)
✓ Data API Tree Cover Loss
  URL: https://data-api.globalforestwatch.org/dataset/umd_tree_cover_loss/latest
  Status: 200
  Keys: ['data', 'status']

✓ Data API Tree Cover Loss
  URL: https://data-api.globalforestwatch.org/dataset/umd_tree_cover_loss/latest
  Status: 200
  Keys: ['data', 'status']

⚠ Analysis Glad: HTTP 403
⚠ Analysis Glad: HTTP 403
⚠ Analysis TC Loss: HTTP 403

Found 1 working endpoint(s)

Working endpoints details:

1. Data API Tree Cover Loss
   URL: https://data-api.globalforestwatch.org/dataset/umd_tree_cover_loss/latest
   Sample data structure:
   {
  "data": {
    "created_on": "2025-05-14T21:39:3

In [25]:
# Test querying the Data API Tree Cover Loss dataset
print("Testing Data API queries for tree cover loss data...")
print("="*60)

import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()

# The dataset exists, now let's try to query it with API key
base_url = "https://data-api.globalforestwatch.org/dataset/umd_tree_cover_loss/latest"
api_key = os.getenv('GFW_API_KEY')

print(f"Using API key: {api_key[:20]}..." if api_key else "No API key found!")

# Try different query approaches
print("\n1. Testing simple query with SQL + API key...")
try:
    # GFW Data API uses SQL queries and requires API key
    sql_query = "SELECT iso, umd_tree_cover_loss__year, SUM(umd_tree_cover_loss__ha) as loss_ha FROM data WHERE iso IN ('BRA', 'USA', 'IDN') AND umd_tree_cover_loss__year >= 2020 GROUP BY iso, umd_tree_cover_loss__year ORDER BY iso, umd_tree_cover_loss__year LIMIT 20"
    
    query_url = f"{base_url}/query"
    params = {'sql': sql_query}
    headers = {'x-api-key': api_key} if api_key else {}
    
    response = requests.get(query_url, params=params, headers=headers, timeout=30)
    print(f"   Status: {response.status_code}")
    
    if response.status_code == 200:
        data = response.json()
        print("   ✓ Query successful!")
        print(f"   Response keys: {list(data.keys())}")
        if 'data' in data:
            print(f"   Records returned: {len(data['data'])}")
            print(f"   Sample record: {json.dumps(data['data'][0] if data['data'] else {}, indent=2)}")
    else:
        print(f"   Response: {response.text[:300]}")
        
except Exception as e:
    print(f"   Error: {str(e)}")

print("\n2. Testing with summary endpoint...")
try:
    sql_query = "SELECT iso, umd_tree_cover_loss__year, SUM(umd_tree_cover_loss__ha) as total_loss FROM data WHERE iso = 'BRA' AND umd_tree_cover_loss__year BETWEEN 2015 AND 2024 GROUP BY iso, umd_tree_cover_loss__year"
    
    query_url = f"{base_url}/query"
    params = {'sql': sql_query}
    headers = {'x-api-key': api_key} if api_key else {}
    
    response = requests.get(query_url, params=params, headers=headers, timeout=30)
    print(f"   Status: {response.status_code}")
    
    if response.status_code == 200:
        data = response.json()
        print("   ✓ Query successful!")
        if 'data' in data and len(data['data']) > 0:
            print(f"   Records: {len(data['data'])}")
            print(f"   Years covered: {[r.get('umd_tree_cover_loss__year') for r in data['data'][:5]]}")
    else:
        print(f"   Response: {response.text[:300]}")
        
except Exception as e:
    print(f"   Error: {str(e)}")

print("\n3. Testing simple country-level aggregation...")
try:
    # Simpler query - just get totals by country and year
    sql_query = "SELECT iso, umd_tree_cover_loss__year, umd_tree_cover_loss__ha FROM data WHERE iso IN ('BRA', 'USA') LIMIT 10"
    
    query_url = f"{base_url}/query"
    params = {'sql': sql_query}
    headers = {'x-api-key': api_key} if api_key else {}
    
    response = requests.get(query_url, params=params, headers=headers, timeout=30)
    print(f"   Status: {response.status_code}")
    print(f"   Full URL: {response.url}")
    
    if response.status_code == 200:
        data = response.json()
        print("   ✓ Simple query works!")
        print(f"   Sample data:")
        print(json.dumps(data, indent=2)[:800])
    else:
        print(f"   Response: {response.text[:500]}")
        
except Exception as e:
    print(f"   Error: {str(e)}")

print("\n" + "="*60)

Testing Data API queries for tree cover loss data...
Using API key: f6130c1a-2af2-4f06-9...

1. Testing simple query with SQL + API key...
   Status: 422
   Response: {"status":"failed","message":"Raster tile set queries require a geometry."}

2. Testing with summary endpoint...
   Status: 422
   Response: {"status":"failed","message":"Raster tile set queries require a geometry."}

2. Testing with summary endpoint...
   Status: 422
   Response: {"status":"failed","message":"Raster tile set queries require a geometry."}

3. Testing simple country-level aggregation...
   Status: 422
   Response: {"status":"failed","message":"Raster tile set queries require a geometry."}

3. Testing simple country-level aggregation...
   Status: 422
   Full URL: https://data-api.globalforestwatch.org/dataset/umd_tree_cover_loss/v1.12/query/json?sql=SELECT+iso%2C+umd_tree_cover_loss__year%2C+umd_tree_cover_loss__ha+FROM+data+WHERE+iso+IN+%28%27BRA%27%2C+%27USA%27%29+LIMIT+10
   Response: {"status":"failed"

In [ ]:
import requests

response = requests.get(
    "https://data-api.globalforestwatch.org/datasets",
    params={
        "page[number]": 1,
        "page[size]": 400
    }
)



print(f"Status Code: {response.status_code}")
for line in response.json()["data"]:
    print(f"Dataset: {line['dataset']}")

Status Code: 200
Dataset: aqueduct_crop_baseline_2020
Dataset: arg_native_forest_land_plan
Dataset: arg_otbn_forest_loss
Dataset: berkeley_earth_temp_anomaly_2000_2020
Dataset: birdlife_alliance_for_zero_extinction_sites
Dataset: birdlife_biodiversity_intactness
Dataset: birdlife_biodiversity_significance
Dataset: birdlife_endemic_bird_areas
Dataset: birdlife_key_biodiversity_areas
Dataset: carbonflux_adm1_change
Dataset: carbonflux_adm1_summary
Dataset: carbonflux_adm2_change
Dataset: carbonflux_adm2_summary
Dataset: carbonflux_iso_change
Dataset: carbonflux_iso_summary
Dataset: cartocritica_mex_protected_areas_2016
Dataset: ci_biodiversity_hotspots
Dataset: cifor_peatlands
Dataset: cities_boundaries_test
Dataset: clark_labs_tropical_pond_aquaculture_1999
Dataset: clark_labs_tropical_pond_aquaculture_2014
Dataset: clark_labs_tropical_pond_aquaculture_2018
Dataset: clark_labs_tropical_pond_aquaculture_change_1999_2014
Dataset: clark_labs_tropical_pond_aquaculture_change_1999_2018
Datas

In [14]:
import requests

response = requests.get(
    "https://data-api.globalforestwatch.org/dataset/umd_tree_cover_loss/v1.9.1/fields"
)

print(f"Status Code: {response.status_code}")
for line in response.json()["data"]:
    print(line)

Status Code: 200
{'pixel_meaning': 'area__ha', 'unit': None, 'description': None, 'statistics': None, 'values_table': None, 'data_type': None, 'compression': None, 'no_data_value': None}
{'pixel_meaning': 'latitude', 'unit': None, 'description': None, 'statistics': None, 'values_table': None, 'data_type': None, 'compression': None, 'no_data_value': None}
{'pixel_meaning': 'longitude', 'unit': None, 'description': None, 'statistics': None, 'values_table': None, 'data_type': None, 'compression': None, 'no_data_value': None}
{'pixel_meaning': 'is__gmw_global_mangrove_extent_1996', 'unit': None, 'description': None, 'statistics': None, 'values_table': {'rows': [{'value': 1, 'meaning': 'true'}], 'default_meaning': 'false'}, 'data_type': None, 'compression': None, 'no_data_value': None}
{'pixel_meaning': 'mapbox_river_basins__id', 'unit': None, 'description': None, 'statistics': None, 'values_table': {'rows': [{'value': 3002, 'meaning': 'Magdalena'}, {'value': 3003, 'meaning': 'Orinoco'}, {'

In [22]:
import os
from dotenv import load_dotenv

load_dotenv()

response = requests.post(  # Changed from GET to POST
    "https://data-api.globalforestwatch.org/dataset/umd_tree_cover_loss/v1.9.1/query/json",
    headers={
        "x-api-key": os.getenv('GFW_API_KEY'),
        "Content-Type": "application/json"
    },
    json={  # Changed from data= to json=
        "sql": "SELECT SUM(area__ha) FROM results WHERE umd_tree_cover_loss__year = 2014",
        "geometry": {
            "type": "Polygon",
            "coordinates": [[
                [103.19732666015625, 0.5537709801264608],
                [103.24882507324219, 0.5647567848663363],
                [103.21277618408203, 0.5932511181408705],
                [103.19732666015625, 0.5537709801264608]
            ]]
        }
    }
)

print(f"status code: {response.status_code}")
if response.status_code == 200:
    print(response.json())
else:
    print(f"Error: {response.text}")

status code: 200
{'data': [{'area__ha': 746.80969}], 'status': 'success'}
